In [1]:
import networkx as nx

# Create an empty graph
G = nx.Graph()

# Add nodes
nodes = ["TextMessage", "Zach", "Justin", "Michelle", "Amy"]
G.add_nodes_from(nodes)

# Add edges with attribute 'HaveAccess'
edges = [
    ("Zach", "TextMessage", {"HaveAccess": 1}),
    ("Justin", "TextMessage", {"HaveAccess": 1}),
    ("Michelle", "TextMessage", {"HaveAccess": 1}),
    ("Amy", "TextMessage", {"HaveAccess": 1})
]
G.add_edges_from(edges)

In [2]:
import pyreason as pr
from pyreason import Threshold

user_defined_thresholds = [
        Threshold("greater_equal", ("number", "total"), 1),
        Threshold("greater_equal", ("percent", "total"), 100),
]

pr.add_rule(pr.Rule('ViewedByAll(x) <- HaveAccess(x,y), Viewed(y)', 'viewed_by_all_rule', user_defined_thresholds))

In [3]:
pr.add_fact(pr.Fact("seen-fact-zach", "Zach", "Viewed", [1, 1], 0, 0, static=True))
pr.add_fact(pr.Fact("seen-fact-justin", "Justin", "Viewed", [1, 1], 0, 0, static=True))
pr.add_fact(pr.Fact("seen-fact-michelle", "Michelle", "Viewed", [1, 1], 1, 1, static=True))
pr.add_fact(pr.Fact("seen-fact-amy", "Amy", "Viewed", [1, 1], 2, 2, static=True))

In [4]:
# Test if the simple program works with thresholds defined
import pyreason as pr
from pyreason import Threshold


def test_custom_thresholds():
    # Reset PyReason
    pr.reset()
    pr.reset_rules()

    # Modify the paths based on where you've stored the files we made above
    graph_path = "group_chat_graph.graphml"

    # Modify pyreason settings to make verbose and to save the rule trace to a file
    pr.settings.verbose = True  # Print info to screen

    # Load all the files into pyreason
    pr.load_graphml(graph_path)

    # add custom thresholds
    user_defined_thresholds = [
        Threshold("greater_equal", ("number", "total"), 1),
        Threshold("greater_equal", ("percent", "total"), 100),
    ]

    pr.add_rule(
        pr.Rule(
            "ViewedByAll(x) <- HaveAccess(x,y), Viewed(y)",
            "viewed_by_all_rule",
            custom_thresholds=user_defined_thresholds,
        )
    )

    pr.add_fact(pr.Fact("seen-fact-zach", "Zach", "Viewed", [1, 1], 0, 3))
    pr.add_fact(pr.Fact("seen-fact-justin", "Justin", "Viewed", [1, 1], 0, 3))
    pr.add_fact(pr.Fact("seen-fact-michelle", "Michelle", "Viewed", [1, 1], 1, 3))
    pr.add_fact(pr.Fact("seen-fact-amy", "Amy", "Viewed", [1, 1], 2, 3))

    # Run the program for three timesteps to see the diffusion take place
    interpretation = pr.reason(timesteps=3)

    # Display the changes in the interpretation for each timestep
    dataframes = pr.filter_and_sort_nodes(interpretation, ["ViewedByAll"])
    for t, df in enumerate(dataframes):
        print(f"TIMESTEP - {t}")
        print(df)
        print()

    assert (
        len(dataframes[0]) == 0
    ), "At t=0 the TextMessage should not have been ViewedByAll"
    assert (
        len(dataframes[2]) == 1
    ), "At t=2 the TextMessage should have been ViewedByAll"

    # TextMessage should be ViewedByAll in t=2
    assert "TextMessage" in dataframes[2]["component"].values and dataframes[2].iloc[
        0
    ].ViewedByAll == [
        1,
        1,
    ], "TextMessage should have ViewedByAll bounds [1,1] for t=2 timesteps"

In [16]:
# Modify the paths based on where you've stored the files we made above
graph_path = "group_chat_graph.graphml"

# Modify pyreason settings to make verbose and to save the rule trace to a file
pr.settings.verbose = True  # Print info to screen

# Load all the files into pyreason
pr.load_graphml(graph_path)

interpretation = pr.reason(timesteps=20)

Timestep: 0
Timestep: 1
Timestep: 2

Converged at time: 2
Fixed Point iterations: 6


In [17]:
# Assuming you have already run the reasoning
dataframes = pr.filter_and_sort_nodes(interpretation, ["ViewedByAll", "Viewed", "HaveAccess"])

# Display the relevant results
for t, df in enumerate(dataframes):
    print(f"TIMESTEP - {t}")
    print(df)
    print()

TIMESTEP - 0
     component ViewedByAll      Viewed HaveAccess
0         Zach      [0, 1]  [1.0, 1.0]     [0, 1]
1       Justin      [0, 1]  [1.0, 1.0]     [0, 1]
2  TextMessage  [1.0, 1.0]      [0, 1]     [0, 1]

TIMESTEP - 1
     component ViewedByAll      Viewed HaveAccess
0         Zach      [0, 1]  [1.0, 1.0]     [0, 1]
1       Justin      [0, 1]  [1.0, 1.0]     [0, 1]
2     Michelle      [0, 1]  [1.0, 1.0]     [0, 1]
3  TextMessage  [1.0, 1.0]      [0, 1]     [0, 1]

TIMESTEP - 2
     component ViewedByAll      Viewed HaveAccess
0         Zach      [0, 1]  [1.0, 1.0]     [0, 1]
1       Justin      [0, 1]  [1.0, 1.0]     [0, 1]
2     Michelle      [0, 1]  [1.0, 1.0]     [0, 1]
3          Amy      [0, 1]  [1.0, 1.0]     [0, 1]
4  TextMessage  [1.0, 1.0]      [0, 1]     [0, 1]



Check if "TextMessage" is Viewed by All:

In [18]:
for t, df in enumerate(dataframes):
    if t == 20:  # Check for timestep 2
        if "TextMessage" in df["component"].values:
            print("TextMessage is viewed by all at timestep 2")
        else:
            print("TextMessage is NOT viewed by all at timestep 2")

List All Components Viewed by "Zach":

In [19]:
for t, df in enumerate(dataframes):
    viewed_by_zach = df[df['component'] == 'Zach']
    print(f"Components viewed by Zach at timestep {t}:")
    
    # Print the DataFrame without the index and with spaced columns
    # print(viewed_by_zach.to_string(index=False, justify='left'))
    # print(viewed_by_zach)

Components viewed by Zach at timestep 0:
Components viewed by Zach at timestep 1:
Components viewed by Zach at timestep 2:
